## Step 1: Clone Repository

In [ ]:
%%bash
# Clone the ACT-modification repository
cd /kaggle/working
git clone https://github.com/aryannzzz/ACT-modification.git
cd ACT-modification
echo "Repository cloned successfully!"

## Step 2: Install Dependencies

In [ ]:
%%bash
cd /kaggle/working/ACT-modification

# Install the package in editable mode
pip install -e .

# Install additional dependencies for MetaWorld
pip install metaworld gymnasium mujoco

echo "Dependencies installed!"

## Step 3: Verify GPU and Environment

In [ ]:
import torch
import sys

print(f"Python version: {sys.version}")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA version: {torch.version.cuda}")
    print(f"GPU device: {torch.cuda.get_device_name(0)}")
    print(f"GPU memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

## Step 4: Configure WandB

Login to Weights & Biases using Kaggle Secrets for experiment tracking.

In [ ]:
# WandB Configuration - Login using Kaggle Secrets
import wandb
import os

try:
    from kaggle_secrets import UserSecretsClient
    secrets = UserSecretsClient()
    wandb_key = secrets.get_secret("WANDB_API_KEY")

    if wandb_key:
        wandb.login(key=wandb_key)
        print("✅ Logged into Weights & Biases")
    else:
        print("⚠️ WANDB_API_KEY is empty")

except Exception as e:
    print("⚠️ WANDB_API_KEY not found:", e)
    print("To use WandB: Add WANDB_API_KEY in Kaggle Secrets (Add-ons → Secrets)")
    # Disable WandB if key not found
    os.environ['WANDB_MODE'] = 'disabled'

## Step 5: Start Training

This will train the ACT policy with the following configuration:
- 50,000 training steps
- Batch size: 8
- Learning rate: 1e-4
- Image augmentation enabled
- Pretrained ResNet18 backbone
- Chunk size: 50 action steps

In [ ]:
%%bash
cd /kaggle/working/ACT-modification

python src/lerobot/scripts/lerobot_train.py \
  --policy.type=act \
  --policy.repo_id=modified-act-reach-v3 \
  --policy.use_vae=true \
  --policy.vision_backbone=resnet18 \
  --policy.chunk_size=50 \
  --policy.n_action_steps=50 \
  --policy.n_obs_steps=1 \
  --policy.latent_dim=32 \
  --policy.kl_weight=10.0 \
  --policy.dim_model=256 \
  --policy.n_encoder_layers=4 \
  --policy.n_decoder_layers=1 \
  --policy.n_vae_encoder_layers=4 \
  --policy.pretrained_backbone_weights=ResNet18_Weights.IMAGENET1K_V1 \
  --env.type=metaworld \
  --env.task=reach-v3 \
  --dataset.repo_id=aadarshram/metaworld-reach-v3 \
  --dataset.image_transforms.enable=true \
  --steps=50000 \
  --batch_size=8 \
  --optimizer.lr=1e-4 \
  --eval_freq=-1 \
  --save_freq=25000 \
  --log_freq=250 \
  --wandb.enable=true \
  --wandb.project=metaworld-modified-act-reach \
  --wandb.disable_artifact=false \
  --output_dir=outputs/train/modified_act_reach

## Step 6: Download Trained Model

After training completes, download the checkpoints to use locally or for evaluation.

In [ ]:
%%bash
# Create a zip file of the trained model
cd /kaggle/working/ACT-modification/outputs/train
zip -r /kaggle/working/modified_act_reach_checkpoints.zip modified_act_reach/
echo "Checkpoints zipped! Download from /kaggle/working/modified_act_reach_checkpoints.zip"

## Step 7: Evaluate the Model (Optional)

You can evaluate the trained model on 50 episodes.

In [ ]:
%%bash
cd /kaggle/working/ACT-modification

python src/lerobot/scripts/lerobot_eval.py \
  --policy.path=outputs/train/modified_act_reach/checkpoints/050000/pretrained_model \
  --env.type=metaworld \
  --env.task=reach-v3 \
  --eval.batch_size=1 \
  --eval.n_episodes=50 \
  --policy.device=cuda

## Training Details

**Model Architecture:**
- Policy: ACT (Action Chunking Transformer) with VAE
- Vision Backbone: ResNet18 (pretrained on ImageNet)
- Transformer: 4 encoder layers, 1 decoder layer
- Latent dimension: 32
- Model dimension: 256

**Training Configuration:**
- Total steps: 50,000
- Batch size: 8
- Learning rate: 1e-4
- Image augmentation: Enabled (ColorJitter, RandomAffine, SharpnessJitter)
- Checkpoints saved every 25,000 steps

**Dataset:**
- Task: MetaWorld reach-v3
- Episodes: 50
- Frames: 2,370
- Repo: aadarshram/metaworld-reach-v3

**Expected Results:**
- Previous baseline: 26% success rate (10k steps)
- Target: Improved success rate with 5x more training